# Autograd

Autograd : Automatic differentiation

Autograd를 사용하면 backprop을 위한 미분 값을 자동으로 게산해줍니다.
자동 계산을 위해서 사용하는 변수는 **torch.autograd에 있는 Variable** 입니다.

먼저 기본세팅을 하겠습니다. import는 다음과 같이 합니다.

In [1]:
import torch
from torch.autograd import Variable
import numpy

In [2]:
a1 = torch.ones(2,2)
print(a1)


 1  1
 1  1
[torch.FloatTensor of size 2x2]



***

# Variable

Variable의 표현은 기본표현과 requires_grad를 포함한 표현으로 총 두 가지가 있습니다.

### 기본표현

In [3]:
a1 = Variable(a1)

### requires_grad를 포함한 표현(왠만하면 requires_grad를 쓰는 것을 추천해드립니다.)

In [4]:
a1 = Variable(a1, requires_grad=True)

여기서 requires_grad는 **'a값에 대한 gradient값이 필요하다.'** 라는 의미입니다.
나중에 CNN, RNN을 구성할 때 필요한 필터들 같은 경우 이미 w값을 가지고 있습니다.
자동으로 requires_grad를 품고 있습니다. 따라서 따로 선언 해줄 필요는 없습니다.
**하지만 기본적인 변수인 a1는 따로 requires_grad를 해주지 않으면
gradient값을 생성하지 않습니다.**
또한 **Variable(a1)만 하게 된다면 나중에 backward 진행할 시 에러가 납니다.**

In [5]:
print(a1)

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]



### autograd.Variable의 의미

autograd.Variable은 총 세 가지를 포함합니다.

* data
    * 위에서 torch.ones(2,2)의 형태로써 Tensor형태의 데이터가 담깁니다.
* grad
    * Data가 거쳐온 layer에 대한 미분값이 축적됩니다. 
    * 업데이트를 위한 gradient descent value이 들어갑니다. 
    * autograd.Variable으로 표현을 한 변수**만** 이 부분에 gradient descent 값이 남습니다.
* grad_fn
    * 미분 값을 계산한 함수에 대한 정보를 담습니다. '어떤 연산에 대한 backward를 진행을 했다' 라는 표현으로 설명할 수 있습니다.

In [6]:
print("- - - - - - - - - - a1.data - - - - - - - - - -")
print(a1.data)

print("- - - - - - - - - - a1.grad - - - - - - - - - -")
print(a1.grad)

print("- - - - - - - - - - a1.grad_fn - - - - - - - - - -")
print(a1.grad_fn)

- - - - - - - - - - a1.data - - - - - - - - - -

 1  1
 1  1
[torch.FloatTensor of size 2x2]

- - - - - - - - - - a1.grad - - - - - - - - - -
None
- - - - - - - - - - a1.grad_fn - - - - - - - - - -
None


In [7]:
b1 = a1+2
print(b1)

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]



In [8]:
c1 = b1**2
print(c1)

Variable containing:
 9  9
 9  9
[torch.FloatTensor of size 2x2]



In [9]:
out = c1.sum()
print(out)

Variable containing:
 36
[torch.FloatTensor of size 1]



### backward의 의미

forward : a1 -> b1 -> c1 -> out이고, backward를 통해 out'/a1'('s는 미분)값이 a1.grad에 담기게 됩니다. 그러면 아까 a1.grad의 None값에 어떠한 값이 채워지게 됩니다. 만약 a1 = Variable(a1, requires_grad=True)에서 requires_grad=True를 하지 않으면 이부분에서 에러가 납니다.

In [10]:
out.backward()
print(out)

Variable containing:
 36
[torch.FloatTensor of size 1]



***

# backward 실행 후, Variable의 의미 확인하기

### a1

In [11]:
print("- - - - - - - - - - a1.data - - - - - - - - - -")
print(a1.data)

print("- - - - - - - - - - a1.grad - - - - - - - - - -")
'''
이 부분이 채워집니다.
'''
print(a1.grad) 

print("- - - - - - - - - - a1.grad_fn - - - - - - - - - -")
print(a1.grad_fn)

- - - - - - - - - - a1.data - - - - - - - - - -

 1  1
 1  1
[torch.FloatTensor of size 2x2]

- - - - - - - - - - a1.grad - - - - - - - - - -
Variable containing:
 6  6
 6  6
[torch.FloatTensor of size 2x2]

- - - - - - - - - - a1.grad_fn - - - - - - - - - -
None


### b1

In [12]:
print("- - - - - - - - - - b1.data - - - - - - - - - -")
print(b1.data)

print("- - - - - - - - - - b1.grad - - - - - - - - - -")
'''
gradient부분이 필요로 하지 않았기 때문에 None인 것입니다. 
이전에 Variable로 한 것이 a1밖에 없었기 때문에 a1만 나오는 것입니다.
'''
print(b1.grad) 

print("- - - - - - - - - - b1.grad_fn - - - - - - - - - -")
'''
이 부분이 채워집니다.
b = a+2였기 때문에 'AddBackward 연산을 했다' 라는 표현으로 볼 수 있습니다.
'''
print(b1.grad_fn) 

- - - - - - - - - - b1.data - - - - - - - - - -

 3  3
 3  3
[torch.FloatTensor of size 2x2]

- - - - - - - - - - b1.grad - - - - - - - - - -
None
- - - - - - - - - - b1.grad_fn - - - - - - - - - -


### c1

In [13]:
print("- - - - - - - - - - c1.data - - - - - - - - - -")
print(c1.data)

print("- - - - - - - - - - c1.grad - - - - - - - - - -")
print(c1.grad) 

print("- - - - - - - - - - c1.grad_fn - - - - - - - - - -")
print(c1.grad_fn)

- - - - - - - - - - c1.data - - - - - - - - - -

 9  9
 9  9
[torch.FloatTensor of size 2x2]

- - - - - - - - - - c1.grad - - - - - - - - - -
None
- - - - - - - - - - c1.grad_fn - - - - - - - - - -


### out

In [14]:
print("- - - - - - - - - - out.data - - - - - - - - - -")
print(out.data)

print("- - - - - - - - - - out.grad - - - - - - - - - -")
print(out.grad) 

print("- - - - - - - - - - out.grad_fn - - - - - - - - - -")
print(out.grad_fn)

- - - - - - - - - - out.data - - - - - - - - - -

 36
[torch.FloatTensor of size 1]

- - - - - - - - - - out.grad - - - - - - - - - -
None
- - - - - - - - - - out.grad_fn - - - - - - - - - -


***

# 한번에 정리해보자

In [15]:
x = torch.ones(3,3)
x = Variable(x, requires_grad=True)

In [16]:
y = x**2
z = y*3

print(z)

Variable containing:
 3  3  3
 3  3  3
 3  3  3
[torch.FloatTensor of size 3x3]



In [17]:
grad = torch.Tensor([0.1,1,10])
print(grad)


  0.1000
  1.0000
 10.0000
[torch.FloatTensor of size 3]



In [18]:
z.backward(grad) 

z = 3 * x^2
미분하면 z'/x' = 3 * 2 * x
위에서 x가 'x = torch.ones(3,3)' 였으므로, 1을 대입하면 z'/x' = 6입니다.
여기서 z.backward(grad)를 하면 (tf에서 feet_dict처럼 대입을 하여) 6에 대해서 각각 곱해집니다.
'grad는 반드시 x와 shape이 동일해야 합니다.'

In [19]:
print(x.data)
print(x.grad)
print(x.grad_fn)


 1  1  1
 1  1  1
 1  1  1
[torch.FloatTensor of size 3x3]

Variable containing:
  0.6000   6.0000  60.0000
  0.6000   6.0000  60.0000
  0.6000   6.0000  60.0000
[torch.FloatTensor of size 3x3]

None


In [20]:
print(z)

Variable containing:
 3  3  3
 3  3  3
 3  3  3
[torch.FloatTensor of size 3x3]

